In [5]:
import json
import random
import os

In [42]:
# # 读取 JSON 文件
# with open("/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_element_0.json", "r", encoding="utf-8") as file:
#     data = json.load(file)
# 读取 JSON 文件
with open("/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_element_3.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [47]:
first_item = data[2]
first_item

{'set_id': '3-14-1-1',
 'lie_id': '1-1',
 'conv_id': 14,
 'truth_id': 1,
 'lie_type': 'altruistic_white_lies',
 'emotion': 'sad',
 'topic': {'scenario': "chatting casually about Eugenia's summer plan to learn piano",
  'situation_topic': 'Eugenia is taking violion courses previously',
  'situation': 'Eugenia is excited to share the dreaming of becoming an musician',
  'lie_objective': "Eugeniaoost Eugenia's confidence and enthusiasm",
  'leave_reason_B': 'feed the dog',
  'leave_reason_D_1': 'need to quickly verify something important',
  'leave_reason_D_2': 'have an unexpected visitor at the door'},
 'relationship': 'families (parents, one kid, aunt/uncle)',
 'multiple_liar': True,
 'multiple_target': False,
 'real_reason_type': True,
 'characters': {'liar': 'Drucilla',
  'target': 'Eugenia',
  'accomplice': 'Marycruz',
  'observer': 'Katey'},
 'lie': {'real_reason_c': "parent Drucilla and Marycruz want support Eugenia's confidence and encourage Eugenia to stay interested in music, ev

In [45]:
# todo:
# 1. 类别123的下面的三个都要重新写：
real_reason_q = "Alice’s social battery is really running dry these days, and she thinks she needs more time to be alone."
lie_q = "Alice has a meeting in the afternoon, so she won’t be able to join for studying."
truth_q = "Alice is actually free for the whole afternoon."
# 2. 每一类的justification question的生成

### 1. comprehensionQA & justificationQA

In [50]:
def generate_comprehensionQA(data):
    liar = data["characters"]["liar"]
    accomplice = data["characters"]["accomplice"]
    target = data["characters"]["target"]

    lie = (data["lie"].get("lie_q") or data["lie"]["lie_c"]).rstrip(".")

    if data.get("multiple_liar") is True:
        question = f"Was what {liar} and {accomplice} told {target}, '{lie}', true?"
    else:
        question = f"Was what {liar} told {target}, '{lie}', true?"
    comprehension_qa = {
        "question": question,
        "question_type": "comprehension",
        "correct_answer": "No",
        "wrong_answer": "Yes"
    }
    
    return {"comprehensionQA": [comprehension_qa]}  # **返回 JSON 结构**

In [51]:
print(json.dumps(generate_comprehensionQA(first_item), indent=4))

{
    "comprehensionQA": [
        {
            "question": "Was what Drucilla and Marycruz told Eugenia, 'Eugenia is very talented', true?",
            "question_type": "comprehension",
            "correct_answer": "No",
            "wrong_answer": "Yes"
        }
    ]
}


In [52]:
def load_justification_options(file_path):
    """
    从指定的 JSON 文件中加载各 set_id 对应的 justification 选项集
    """
    with open(file_path, "r", encoding="utf-8") as f:
        options_by_set = json.load(f)
    return options_by_set

In [57]:
# freefrom // mcq
def generate_justificationQA(data):
    set_id = data["set_id"]
    # options_by_set = load_justification_options("/Users/liuyiwei/Desktop/TactfulToM/dataset/justification_option.json")
    # options = options_by_set.get(set_id)
    # if options is None:
    #     raise ValueError(f"Options not found: {set_id}")
    
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    
    # lie = data["lie"]["lie_c"].rstrip(".")
    # real_reason = data["lie"]["real_reason_c"].rstrip(".")
    real_reason = (data["lie"].get("real_reason_q") or data["lie"]["real_reason_c"]).rstrip(".")
    truth = data["lie"].get("truth_q") or data["lie"]["truth_c"].rstrip(".")
    lie = (data["lie"].get("lie_q") or data["lie"]["lie_c"]).rstrip(".")

    if data.get("multiple_liar") is True:
        question_lie = f"Why did {liar} and {accomplice} tell {target} '{lie}'?"
        question_real_reason = f"Why did {liar} and {accomplice} tell {target} the '{lie}' instead of the '{real_reason}'?"
    else:
        question_lie = f"Why did {liar} tell {target} '{lie}'?"
        question_real_reason = f"Why did {liar} tell {target} the '{lie}' instead of the '{real_reason}'?"
        
    justification_qas = [
        {
            "question": question_lie,
            "question_type": "justification:lie",
            # "correct_answer":options["truth"]["correct_answer"],
            # "wrong_answer": options["truth"]["wrong_answer"]
        },
        # 这个问题的核心变成强调了 lie vs real reason 的对比，突出强调了说谎者的动机中存在一个真实的但不便明说的原因（即真实情绪或心理原因）。因此，在设计选项时，需要稍作调整，突出 避免负面影响、礼貌、维护关系 等。
        {   
            "question": question_real_reason,
            "question_type": "justification:real_reason",
            # "correct_answer":options["real_reason"]["correct_answer"],
            # "wrong_answer": options["real_reason"]["wrong_answer"]
        }
    ]

    return {"justificationQA": justification_qas}  # 返回 JSON 结构

In [58]:
print(json.dumps(generate_justificationQA(data[0]), indent=4))

{
    "justificationQA": [
        {
            "question": "Why did Belinda and Leo tell Ayra 'Ayra's drawing is really creative and Ayra is getting very talented'?",
            "question_type": "justification:lie"
        },
        {
            "question": "Why did Belinda and Leo tell Ayra the 'Ayra's drawing is really creative and Ayra is getting very talented' instead of the 'Belinda and Leo want support Ayra's confidence and encourage Ayra to stay interested in their hobby even when Ayra is not good at it'?",
            "question_type": "justification:real_reason"
        }
    ]
}


### 2. fact_QA

In [63]:
# mcq
# dist
def generate_fact_QA(data):
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]
    
    real_reason = (data["lie"].get("real_reason_q") or data["lie"]["real_reason_c"]).rstrip(".")
    truth = data["lie"].get("truth_q") or data["lie"]["truth_c"].rstrip(".")
    lie = (data["lie"].get("lie_q") or data["lie"]["lie_c"]).rstrip(".")
    # real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # truth =  data["lie"]["truth_c"]
    
    truth_id = data["truth_id"]

    # Because the correct answer has liar, make the incorrect choices all have liar and acomplice as well
    # anyway, is to make the incorrect choices a little more confusing
    # f"{observer}, {accomplice} and {target} talked about that.",
    # f"{observer} and {target} talked about that",
    # f"{observer}, {target} and {liar} talked about that.",
    wrong_answer_reason = [
        f"{accomplice} and {target} talked about that.",
        f"{observer} and {liar} talked about that.",
        f"{observer} and {accomplice} talked about that.",
        f"{target} and {liar} talked about that.",
        f"{target}, {accomplice} and {liar} talked about that.",
        f"{liar}, {observer} and {accomplice} talked about that."
    ]

    random_wrong_answer_reason = random.sample(wrong_answer_reason, 3)

    # **1️⃣ ReasonQA**
    reason_qa = {
        "question": f"Who discussed '{real_reason}'?",
        "question_type": "fact:real_reason",
        "correct_answer": f"{liar} and {accomplice} talked about that.",
        "wrong_answer": random_wrong_answer_reason 
    }

    # **2️⃣ truthQA（only when truth_id == 1）**
    truth_qa = None
    if truth_id == 1:
        truth =  truth.rstrip(".")
        # f"{target} and {liar} talked about that"
        # f"{observer} and {target} talked about that",
        # f"{accomplice} and {target} talked about that",
        wrong_answer_truth = [
            f"{observer}, {accomplice} and {target} talked about that.",
            f"{observer}, {target} and {liar} talked about that.",
            f"{target}, {accomplice} and {liar} talked about that.",
            f"{liar} and {accomplice} talked about that",
            f"{observer} and {liar} talked about that",
            f"{observer} and {accomplice} talked about that",
        ]
        random_wrong_answer_truth = random.sample(wrong_answer_truth, 3)

        truth_qa = {
            "question": f"Who discussed '{truth}'?",
            "question_type": "fact:truth",
            "correct_answer": f"{liar}, {observer} and {accomplice} talked about that.",
            "wrong_answer": random_wrong_answer_truth
            }

    return {"fact_reasonQA": [reason_qa], "fact_truthQA": [truth_qa] if truth_qa else []}

In [65]:
print(json.dumps(generate_fact_QA(data[0]), indent=4))

{
    "fact_reasonQA": [
        {
            "question": "Who discussed 'Belinda and Leo want support Ayra's confidence and encourage Ayra to stay interested in their hobby even when Ayra is not good at it'?",
            "question_type": "fact:real_reason",
            "correct_answer": "Belinda and Leo talked about that.",
            "wrong_answer": [
                "Ayra and Belinda talked about that.",
                "Ayra, Leo and Belinda talked about that.",
                "Leo and Ayra talked about that."
            ]
        }
    ],
    "fact_truthQA": [
        {
            "question": "Who discussed 'Ayra is not talented for drawing at all'?",
            "question_type": "fact:truth",
            "correct_answer": "Belinda, Mia and Leo talked about that.",
            "wrong_answer": [
                "Mia and Belinda talked about that",
                "Mia and Leo talked about that",
                "Mia, Leo and Ayra talked about that."
            ]
        }
  

### 3. beliefQAs

In [19]:
# The text provides no information on 'What does {liar} believe about '{real_reason_question}'. Therefore, it can be stated that {liar} is unaware about this information.

# {liar} is unaware that who discussed it.

# Jaiden believes that Veronica is unaware that the strategies Christian uses to manage stress in a high-pressure work environment, because Veronica was not involved in the conversation where it was discussed.


In [20]:
# 一阶的
# A is believes that 
#   can be the true 
#   or wrong one

# A is unaware that xx
#   The text provide no information on xx, so it can be stated that {liar} is unaware about xx.
#   Based on the text, xxx

In [21]:
# 二阶的
# A believes B believes that (can be the true one or wrong one)
# A believes B is unaware that xxx
# A is unaware that B believes that (can be the true one or wrong one)
# A is unaware that what believes of.
# the text provide no information on xx, it can be stated that {liar} is unaware about this information.

In [22]:
# def (option, prefix="The text did not provide enough information about this, so we can state that "):
#     """
#     随机决定是否在传入的字符串前加上前缀。

#     参数：
#     - option: 原始选项字符串。
#     - prefix: 前缀字符串，默认为 "The text did not provide enough information about this, so we can state that "。

#     返回：
#     - 根据随机结果返回原始选项或加上前缀后的选项。
#     """
#     # if random.choice([True, False]):
#     option = prefix + option
#     return option

In [66]:
def generate_1stbeliefQAs(data):
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]
    
    # real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # # truth =  data["lie"]["truth_c"].rstrip(".")
    # truth = (data["lie"].get("truth_c") or "").rstrip(".")
    real_reason = (data["lie"].get("real_reason_q") or data["lie"]["real_reason_c"]).rstrip(".")
    truth = data["lie"].get("truth_q") or data["lie"]["truth_c"].rstrip(".")
    lie = (data["lie"].get("lie_q") or data["lie"]["lie_c"]).rstrip(".")

    truth_id = data["truth_id"]
    truth_question = f"Who discussed '{truth}'?" if truth_id == 1 else None
    real_reason_question = f"Who discussed '{real_reason}'?"

    belief_qas = []

    #######################################
    # **first-order belief（real reason）**
    # {liar}
    # +-
    first_wrong_options_liar = [
        f"{liar} believes that {target} discussed {real_reason}.",
        f"{liar} believes that {observer} discussed {real_reason}.",
        f"{liar} believes that {observer} and {target} discussed {real_reason}.",
        f"{liar} believes that {accomplice} and {target} discussed {real_reason}.",
        f"{liar} believes that {observer} and {accomplice} discussed {real_reason}.",
    ]
    # -+
    second_wrong_option_liar = f"{liar} is unaware that {accomplice} discussed {real_reason}."
    # --
    third_wrong_options_liar = [
        f"{liar} is unaware that {target} discussed {real_reason}.",
        f"{liar} is unaware that {observer} discussed {real_reason}.",
        f"{liar} is unaware that {target} and {observer} discussed {real_reason}.",
        f"{liar} is unaware that {accomplice} and {observer} discussed {real_reason}.",
        f"{liar} is unaware that {target} and {accomplice} discussed {real_reason}.",
    ]
    belief_qas.append({
        "question": f"What does {liar} believe about '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "first-order:A",
        "correct_answer": f"{liar} believes that {accomplice} discussed {real_reason}.", # ++
        "wrong_answer": [
                # +-
                random.choice(first_wrong_options_liar),
                # -+
                second_wrong_option_liar,
                # --
                random.choice( third_wrong_options_liar)
                ],
        "real_reason_accessibility": "accessible"
    })

    # {target}
    # +-
    first_wrong_options_target = [
        f"{target} is unaware that {observer} discussed {real_reason}.",
        f"{target} is unaware that {liar} and {observer} discussed {real_reason}.",
        f"{target} is unaware that {accomplice} and {observer} discussed {real_reason}."
    ]
    # -+
    second_wrong_option_target = [
        f"{target} believes that {liar} and {accomplice} discussed {real_reason}.",
        f"{target} believes that {liar} discussed {real_reason}.",
        f"{target} believes that {accomplice} discussed {real_reason}."
    ]
    # --
    third_wrong_options_target = [
        f"{target} believes that {accomplice}, {liar} and {observer} discussed {real_reason}.",
        f"{target} believes that {accomplice} and {observer} discussed {real_reason}.",
        f"{target} believes that  and {observer} discussed {real_reason}.",
        f"{target} believes that {observer} discussed {real_reason}."
    ]

    belief_qas.append({
        "question": f"What does {target} believe about '{real_reason_question}'?",
        "question_type": "tom:belief:inaccessible:reason",
        "tom_type": "first-order:B",
        "correct_answer": f"{target} is unaware that {liar} and {accomplice} discussed {real_reason}.",
        "wrong_answer": [
            # +-
            random.choice(first_wrong_options_target),
            # -+
            random.choice(second_wrong_option_target),
            # --
            random.choice(third_wrong_options_target)
        ],
        "real_reason_accessibility": "inaccessible"
    })

    # {accomplice}
    # +-
    first_wrong_options_accomplice = [
        f"{accomplice} believes that {observer} discussed {real_reason}.",
        f"{accomplice} believes that {target} discussed {real_reason}.",
        f"{accomplice} believes that {observer} and {target} discussed {real_reason}.",
        f"{accomplice} believes that {observer} and {liar} discussed {real_reason}.",
        f"{accomplice} believes that {liar} and {target} discussed {real_reason}.",
        f"{accomplice} believes that {liar}, {observer} and {target} discussed {real_reason}."
    ]
    # -+
    second_wrong_option_accomplice = f"{accomplice} is unaware that {liar} discussed {real_reason}."
    # --
    third_wrong_options_accomplice = [
        f"{accomplice} is unaware that {observer} discussed {real_reason}.",
        f"{accomplice} is unaware that {target} discussed {real_reason}.",
        f"{accomplice} is unaware that {observer} and {target} discussed {real_reason}.",
        f"{accomplice} is unaware that {observer} and {liar} discussed {real_reason}.",
        f"{accomplice} is unaware that {liar} and {target} discussed {real_reason}.",
        f"{accomplice} is unaware that {liar}, {observer} and {target} discussed {real_reason}."
    ]
    belief_qas.append({
        "question": f"What does {accomplice} believe about '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "first-order:C",
        "correct_answer": f"{accomplice} believes that {liar} discussed {real_reason}.", # ++
        "wrong_answer": [
                # +-
                random.choice(first_wrong_options_accomplice),
                # -+
                (second_wrong_option_accomplice),
                # --
                (random.choice(third_wrong_options_accomplice))
                ],
        "real_reason_accessibility": "accessible"
    })
    #######################################

    #######################################
    # **第一阶信念（关于 truth_question）**
    # **如果 truthQA 不是空的，加入关于 truth_question 的 belief question**
    if truth_id == 1 and truth_question:
        # {liar}
        # +-
        first_wrong_options_liar_truth = [
            f"{liar} believes that {target} discussed {truth}.",
            f"{liar} believes that {observer} and {target} discussed {truth}.",
            f"{liar} believes that {accomplice} and {target} discussed {truth}.",
            f"{liar} believes that {accomplice}, {observer} and {target} discussed {truth}."
        ]
        # -+
        second_wrong_option_liar_truth = [
            f"{liar} is unaware that {accomplice} and {observer} discussed {truth}.",
            f"{liar} is unaware that {accomplice} discussed {truth}.",
            f"{liar} is unaware that {observer} discussed {truth}."
        ]
        # --
        third_wrong_options_liar_truth = [
            f"{liar} is unaware that {target} discussed {truth}.",
            f"{liar} is unaware that {observer} and {target} discussed {truth}.",
            f"{liar} is unaware that {accomplice} and {target} discussed {truth}.",
            f"{liar} is unaware that {accomplice}, {observer} and {target} discussed {truth}."
        ]
        belief_qas.append({
        "question": f"What does {liar} believe about '{truth_question}'?",
        "question_type": "tom:belief:accessible:truth",
        "tom_type": "first-order:A",
        "correct_answer": f"{liar} believes that {accomplice} and {observer} discussed {truth}.",
        "wrong_answer": [
                # +-
                random.choice(first_wrong_options_liar_truth),
                # -+
                random.choice(second_wrong_option_liar_truth),
                # --
                random.choice(third_wrong_options_liar_truth)
                ],
        "truth_accessibility": "accessible"
    })
        
        # {target}
        # +-
        # first_wrong_options_target_truth = [
        #     f"{target} is unaware that {observer} discussed {real_reason}.",
        #     f"{target} is unaware that {liar} and {observer} discussed {real_reason}.",
        #     f"{target} is unaware that {accomplice} and {observer} discussed {real_reason}."
        # ]
        # -+
        second_wrong_option_target_truth = [
            f"{target} believes that {liar}, {accomplice} and {observer} discussed about {truth}.",
            f"{target} believes that {accomplice} and {observer} discussed about {truth}.",
            f"{target} believes that {liar} and {observer} discussed about {truth}.",
            f"{target} believes that {liar} and {accomplice} discussed about {truth}.",
            f"{target} believes that {liar} discussed about {truth}.",
            f"{target} believes that {accomplice} discussed about {truth}.",
            f"{target} believes that {observer} discussed about {truth}.",
        ]
        # --
        # third_wrong_options_target_truth = [
        #     f"{target} believes that {accomplice} and {observer} discussed {real_reason}.",
        #     f"{target} believes that {liar}  and {observer} discussed {real_reason}.",
        #     f"{target} believes that {observer} discussed {real_reason}."
        # ]
        # ++
        correct_answer = f"{target} is unaware that {liar}, {accomplice} and {observer} discussed about {truth}."
        belief_qas.append({
        "question": f"What does {target} believe about '{truth_question}'?",
        "question_type": "tom:belief:inaccessible:truth",
        "tom_type": "first-order:B",
        "correct_answer": (correct_answer),
        "wrong_answer": random.sample(second_wrong_option_target_truth, 3),
        "truth_accessibility": "inaccessible"
    })
        
        # {accomplice}
        # +-
        first_wrong_options_accomplice_truth = [
            f"{accomplice} believes that {target} discussed {truth}.",
            f"{accomplice} believes that {liar} and {target} discussed {truth}.",
            f"{accomplice} believes that {observer} and {target} discussed {truth}.",
            f"{accomplice} believes that {observer}, {liar} and {target} discussed {truth}."
        ]
        # -+
        second_wrong_option_accomplice_truth = [
            f"{accomplice} is unaware that {liar} discussed {truth}.",
            f"{accomplice} is unaware that {observer} discussed {truth}.",
            f"{accomplice} is unaware that {liar} and {observer} discussed {truth}.",
        ]
        # --
        third_wrong_options_accomplice_truth = [
            f"{accomplice} is unaware that {target} discussed {truth}.",
            f"{accomplice} is unaware that {liar} and {target} discussed {truth}.",
            f"{accomplice} is unaware that {observer} and {target} discussed {truth}."
            f"{accomplice} is unaware that {observer}, {liar} and {target} discussed {truth}."
        ]

        belief_qas.append({
        "question": f"What does {target} believe about '{truth_question}'?",
        "question_type": "tom:belief:accessible:truth",
        "tom_type": "first-order:C",
        "correct_answer": f"{accomplice} believe that {liar} and {observer} discussed {truth}.", # ++
        "wrong_answer": [
                # +-
                random.choice(first_wrong_options_accomplice_truth),
                # -+
                random.choice(second_wrong_option_accomplice_truth),
                # --
                random.choice(third_wrong_options_accomplice_truth),
                ],
        "truth_accessibility": "accessible"
    })

    return {"beliefQAs": belief_qas}

In [67]:
print(json.dumps(generate_1stbeliefQAs(data[0]), indent=4))

{
    "beliefQAs": [
        {
            "question": "What does Belinda believe about 'Who discussed 'Belinda and Leo want support Ayra's confidence and encourage Ayra to stay interested in their hobby even when Ayra is not good at it'?'?",
            "question_type": "tom:belief:accessible:reason",
            "tom_type": "first-order:A",
            "correct_answer": "Belinda believes that Leo discussed Belinda and Leo want support Ayra's confidence and encourage Ayra to stay interested in their hobby even when Ayra is not good at it.",
            "wrong_answer": [
                "Belinda believes that Ayra discussed Belinda and Leo want support Ayra's confidence and encourage Ayra to stay interested in their hobby even when Ayra is not good at it.",
                "Belinda is unaware that Leo discussed Belinda and Leo want support Ayra's confidence and encourage Ayra to stay interested in their hobby even when Ayra is not good at it.",
                "Belinda is unaware that 

In [68]:
def generate_2ndbeliefQAs(data):
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]

    # real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # # truth = data["lie"]["truth_c"].rstrip(".")
    # truth = (data["lie"].get("truth_c") or "").rstrip(".")

    real_reason = (data["lie"].get("real_reason_q") or data["lie"]["real_reason_c"]).rstrip(".")
    truth = data["lie"].get("truth_q") or data["lie"]["truth_c"].rstrip(".")
    lie = (data["lie"].get("lie_q") or data["lie"]["lie_c"]).rstrip(".")
    
    truth_id = data["truth_id"]
    truth_question = f"Who discussed '{truth}'?" if truth_id == 1 else None
    real_reason_question = f"Who discussed '{real_reason}'?"

    belief_qas = []
    
    #######################################
    # **第二阶信念（关于 real reason）**
    ########### {liar} ########
    # 还有长答案和短答案
    # Jaiden believes that Veronica is unaware that.
    
    # A不在对话里面 --inaccessible
    # The text provides no information on what Jaiden believes about Veronica's belief regarding xx, Therefore, it can be stated that Jaiden is unaware about this information.
    # A在对话里面，B不在对话里面 --accessible
    # A believes that B is unaware that the strategies Christian uses to manage stress in a high-pressure work environment, because B was not involved in the conversation when this was discussed.",
    
    # AB
    # liar:target
    belief_qas.append({
        "question": f"What does {liar} believe about {target}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order:AB",
        # ++
        "correct_answer": f"{liar} believes that {target} is unaware that {real_reason}, because {target} was not involved in the conversation when this was discussed.",
        "wrong_answer": [
            # +-
            f"{liar} believes that {target} believes that {real_reason}.",
            # -+
            f"The text provides no information on what {liar} believes about {target}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {target} is unaware that {real_reason}.",
            # --
            f"The text provides no information on what {liar} believes about {target}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {target} believes that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })
    # AC
    # liar:accomplice
    # Who discussed 'real_reason'?"
    belief_qas.append({
        "question": f"What does {liar} believe about {accomplice}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order:AC",
        # ++
        "correct_answer": f"{liar} believes that {accomplice} believes that {real_reason}.",
        "wrong_answer": [
            # +-
            f"{liar} believes that {accomplice} is unaware that {real_reason}, because {accomplice} was not involved in the conversation when this was discussed.",
            # -+
            f"The text provides no information on what {liar} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {accomplice} is unaware that {real_reason}.",
            # --
            f"The text provides no information on what {liar} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {accomplice} believes that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })
    # AD
    # liar:observer
    belief_qas.append({
        "question": f"What does {liar} believe about {observer}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order:AD",
        # ++
        "correct_answer":f"{liar} believes that {observer} is unaware that {real_reason}, because {observer} was not involved in the conversation when this was discussed.",
        "wrong_answer": [
            # +-
            f"{liar} believes that {observer} believes that {real_reason}.",
            # -+
            f"The text provides no information on what {liar} believes about {observer}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {target} is unaware that {real_reason}.",
            # --
            f"The text provides no information on what {liar} believes about {observer}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {target} believes that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })

    ########### {target} ###########
    # BA
    # target:liar
    belief_qas.append({
        "question": f"What does {target} believe about {liar}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:inaccessible:reason",
        "tom_type": "second-order:BA",
        # ++
        "correct_answer": f"The text provides no information on what {target} believes about {liar}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {liar} believes that {real_reason}.",
        "wrong_answer": [
            # +-
            f"The text provides no information on what {target} believes about {liar}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {liar} is unaware that {real_reason}.",
            # -+
            f"{target} believes that {liar} believes that {real_reason}.",
            # --
            f"{target} believes that {liar} is unaware that {real_reason}, because {liar} was not involved in the conversation when this was discussed."
        ],
        "real_reason_accessibility": "inaccessible"
    })
    # BC
    # target:accomplice
    belief_qas.append({
        "question": f"What does {target} believe about {accomplice}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:inaccessible:reason",
        "tom_type": "second-order:BC",
        # ++
        "correct_answer": f"The text provides no information on what {target} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {accomplice} believes that {real_reason}.",
        "wrong_answer": [
            # +-
            f"The text provides no information on what {target} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {accomplice} is unaware that {real_reason}.",
            # -+
            f"{target} believes that {accomplice} believes that {real_reason}.",
            # --
            f"{target} believes that {accomplice} is unaware that {real_reason}, because {accomplice} was not involved in the conversation when this was discussed."
        ],
        "missed_info_accessibility": "inaccessible"
    })

    # BD
    # target:observer
    belief_qas.append({
        "question": f"What does {target} believe about {observer}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:inaccessible:reason",
        "tom_type": "second-order:BD",
        # ++
        "correct_answer": f"The text provides no information on what {target} believes about {observer}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {observer} is unaware that {real_reason}.",
        "wrong_answer": [
            # +-
            f"The text provides no information on what {target} believes about {observer}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {observer} believes that {real_reason}.",
            # -+
           f"{target} believes that {observer} is unaware that {real_reason}, because {observer} was not involved in the conversation when this was discussed.",
            # --
            f"{target} believes that {observer} believes that {real_reason}."
        ],
        # "wrong_answer": f"{target} believes that {observer} is unaware that.",
        "missed_info_accessibility": "inaccessible"
    })

    ########### {accomplice} ########
    # CA
    # accomplice:liar
    belief_qas.append({
        "question": f"What does {accomplice} believe about {liar}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order:CA",
        # ++
        "correct_answer": f"{accomplice} believes that {liar} believes that {real_reason}.",
        "wrong_answer": [
            # +-
            f"{accomplice} believes that {liar} is unaware that {real_reason}, because {liar} was not involved in the conversation when this was discussed.",
            # -+
            f"The text provides no information on what {accomplice} believes about {liar}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {liar} is unaware that {real_reason}.",
            # --
            f"The text provides no information on what {accomplice} believes about {liar}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {liar} believes that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })
    # accomplice:target
    # CB
    belief_qas.append({
        "question": f"What does {accomplice} believe about {target}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order:CB",
        # ++
        "correct_answer": f"{accomplice} believes that {target} is unaware that {real_reason}, because {target} was not involved in the conversation when this was discussed.",
        "wrong_answer": [
            # +-
            f"{accomplice} believes that {target} believes that {real_reason}.",
            # -+
            f"The text provides no information on what {accomplice} believes about {target}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {target} is unaware that {real_reason}.",
            # --
            f"The text provides no information on what {accomplice} believes about {target}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {target} believes that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })

    # accomplice:observer
    # CD
    belief_qas.append({
        "question": f"What does {accomplice} believe about {observer}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order:CD",
        # ++
        "correct_answer":f"{accomplice} believes that {observer} is unaware that {real_reason}, because {observer} was not involved in the conversation when this was discussed.",
        "wrong_answer": [
            # +-
            f"{accomplice} believes that {observer} believes that {real_reason}.",
            # -+
            f"The text provides no information on what {accomplice} believes about {observer}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {target} is unaware that {real_reason}.",
            # --
            f"The text provides no information on what {accomplice} believes about {observer}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {target} believes that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })
    
    # ########### {observer} #########
    # # DA
    # # observer:liar
    # belief_qas.append({
    #     "question": f"What does {observer} believe about {liar}'s belief regarding '{real_reason_question}'?",
    #     "question_type": "tom:belief:inaccessible:reason",
    #     "tom_type": "second-order",
    #     "correct_answer": f"{observer} is unaware that {liar}'s regarding that.",
    #     "wrong_answer": f"{observer} believes that {liar} is aware of that.",
    #     "real_reason_accessibility": "inaccessible"
    # })
    # # DB
    # # observer:target
    # belief_qas.append({
    #     "question": f"What does {observer} believe about {target}'s belief regarding '{real_reason_question}'?",
    #     "question_type": "tom:belief:inaccessible:reason",
    #     "tom_type": "second-order",
    #     "correct_answer": f"{observer} is unaware that {target}'s regarding that.",
    #     "wrong_answer": f"{observer} believes that {target} is aware of that.",
    #     "real_reason_accessibility": "inaccessible"
    # })

    # # DC
    # # # observer:accomplice
    # belief_qas.append({
    #     "question": f"What does {observer} believe about {accomplice}'s belief regarding '{real_reason_question}'?",
    #     "question_type": "tom:belief:inaccessible:reason",
    #     "tom_type": "second-order",
    #     "correct_answer": f"{observer} is unaware that {accomplice}'s regarding that.",
    #     "wrong_answer": f"{observer} believes that {accomplice} is aware of that.",
    #     "real_reason_accessibility": "inaccessible"
    # })
    #######################################

    #######################################
    # **第二阶信念（关于 truth）**
    # **如果 truthQA 不是空的，加入关于 truth_question 的二阶信念**
    if truth_id == 1 and truth_question:
        ########### {liar} ########
        # AB
        # liar:target
        belief_qas.append({
            "question": f"What does {liar} believe about {target}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order:AB",
            # ++
            "correct_answer": f"{liar} believes that {target} is unaware that {truth}, because {target} was not involved in the conversation when this was discussed.",
            "wrong_answer": [
                # +-
                f"{liar} believes that {target} believes that {truth}.",
                # -+
                f"The text provides no information on what {liar} believes about {target}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {target} is unaware that {truth}.",
                # --
                f"The text provides no information on what {liar} believes about {target}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {target} believes that {truth}."
            ],
            "truth_accessibility": "accessible"
        })
        # AC
        # liar:accomplice
        belief_qas.append({
            "question": f"What does {liar} believe about {accomplice}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order:AC",
            # ++
            "correct_answer": f"{liar} believes that {accomplice} believes that {truth}.",
            "wrong_answer": [
                # +-
                f"{liar} believes that {accomplice} is unaware that {truth}, because {accomplice} was not involved in the conversation when this was discussed.",
                # -+
                f"The text provides no information on what {liar} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {accomplice} is unaware that {truth}.",
                # --
                f"The text provides no information on what {liar} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {accomplice} believes that {truth}."
            ],
            "truth_accessibility": "accessible"
        })
        # AD
        # liar:observer
        belief_qas.append({
            "question": f"What does {liar} believe about {observer}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order:AD",
            # ++
            "correct_answer":f"{liar} believes that {observer} believes that {truth}.",
            "wrong_answer": [
                # +-
                f"{liar} believes that {observer} is unaware that {truth}, because {observer} was not involved in the conversation when this was discussed.",
                # -+
                f"The text provides no information on what {liar} believes about {observer}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {target} believes that {truth}.",
                # --
                f"The text provides no information on what {liar} believes about {observer}'s belief regarding that. Therefore, it can be stated that {liar} is unaware that {target} is unaware that {truth}."
            ], 
            "wrong_answer": f"{liar} is unaware that {observer}'s belief regarding that.",
            "truth_accessibility": "accessible"
        })
        ########### {target} ###########
        # BA
        # target:liar
        belief_qas.append({
            "question": f"What does {target} believe about {liar}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:inaccessible:truth",
            "tom_type": "second-order:BA",
            # ++
            "correct_answer": f"The text provides no information on what {target} believes about {liar}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {liar} believes that {truth}.",
            "wrong_answer": [
                # +-
                f"The text provides no information on what {target} believes about {liar}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {liar} is unaware that {truth}.",
                # -+
                f"{target} believes that {liar} believes that {truth}.",
                # --
                f"{target} believes that {liar} is unaware that {truth}, because {liar} was not involved in the conversation when this was discussed."
            ],
            "truth_accessibility": "inaccessible"
        })
        # BC
        # target:accomplice
        belief_qas.append({
            "question": f"What does {target} believe about {accomplice}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:inaccessible:truth",
            "tom_type": "second-order:BC",
            # ++
            "correct_answer": f"The text provides no information on what {target} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {accomplice} believes that {truth}.",
            "wrong_answer": [
                # +-
                f"The text provides no information on what {target} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {accomplice} is unaware that {truth}.",
                # -+
                f"{target} believes that {accomplice} believes that {truth}.",
                # --
                f"{target} believes that {accomplice} is unaware that {truth}, because {accomplice} was not involved in the conversation when this was discussed."
            ],
            "truth_accessibility": "inaccessible"
        })

        # BD
        # target:observer
        belief_qas.append({
            "question": f"What does {target} believe about {observer}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:inaccessible:truth",
            "tom_type": "second-order:BD",
            # ++
            "correct_answer": f"The text provides no information on what {target} believes about {observer}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {observer} believes that {truth}.",
            "wrong_answer": [
                # +-
                f"The text provides no information on what {target} believes about {observer}'s belief regarding that. Therefore, it can be stated that {target} is unaware that {observer} is unaware that {truth}.",
                # -+
                f"{target} believes that {observer} believes that {truth}.",
                # --
                f"{target} believes that {observer} is unaware that {truth}, because {observer} was not involved in the conversation when this was discussed."

            ],
            "truth_accessibility": "inaccessible"
        })

        ########### {accomplice} ########
        # CA
        # accomplice:liar
        belief_qas.append({
            "question": f"What does {accomplice} believe about {liar}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order:CA",
            # ++
            "correct_answer": f"{accomplice} believes that {liar} believes that {truth}.",
            "wrong_answer": [
                # +-
                f"{accomplice} believes that {liar} is unaware that {truth}, because {liar} was not involved in the conversation when this was discussed.",
                # -+
                f"The text provides no information on what {accomplice} believes about {liar}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {liar} is unaware that {truth}.",
                # --
                f"The text provides no information on what {accomplice} believes about {liar}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {liar} believes that {truth}."
            ],
            "truth_accessibility": "accessible"
        })
        # accomplice:target
        # CB
        belief_qas.append({
            "question": f"What does {accomplice} believe about {target}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order:CB",
            # ++
            "correct_answer": f"{accomplice} believes that {target} is unaware that {truth}, because {target} was not involved in the conversation when this was discussed.",
            "wrong_answer": [
                # +-
                f"{accomplice} believes that {target} believes that {truth}.",
                # -+
                f"The text provides no information on what {accomplice} believes about {target}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {target} is unaware that {truth}.",
                # --
                f"The text provides no information on what {accomplice} believes about {target}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {target} believes that {truth}."
            ],
            "truth_accessibility": "accessible"
        })

        # accomplice:observer
        # CD
        belief_qas.append({
            "question": f"What does {accomplice} believe about {observer}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order:CD",
            "correct_answer":f"{accomplice} believes that {observer} believes that {truth}.",
            "wrong_answer": [
                # +-
                f"{accomplice} believes that {observer} is unaware that {truth}, because {observer} was not involved in the conversation when this was discussed.",
                # -+
                f"The text provides no information on what {accomplice} believes about {observer}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {target} believes that {truth}.",
                # --
                f"The text provides no information on what {accomplice} believes about {observer}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware that {target} is unaware that {truth}."
            ],
            "truth_accessibility": "accessible"
        })
        # ########### {observer} #########
        # # DA
        # # observer:liar
        # belief_qas.append({
        #     "question": f"What does {observer} believe about {liar}'s belief regarding '{truth_question}'?",
        #     "question_type": "tom:belief:accessible:truth",
        #     "tom_type": "second-order",
        #     "correct_answer": f"{observer} believes that {liar} is aware of that.",
        #     "wrong_answer": f"{observer} is unaware that {liar}'s belief regarding that.",
        #     "truth_accessibility": "accessible"
        # })
        # # DB
        # # observer:target
        # belief_qas.append({
        #     "question": f"What does {observer} believe about {target}'s belief regarding '{truth_question}'?",
        #     "question_type": "tom:belief:accessible:truth",
        #     "tom_type": "second-order",
        #     "correct_answer": f"{observer} believes that {target} is unaware that.",
        #     "wrong_answer": f"{observer} is unaware that {target}'s belief regarding that.",
        #     "truth_accessibility": "accessible"
        # })
        # # DC
        # # # observer:accomplice
        # belief_qas.append({
        #     "question": f"What does {observer} believe about {accomplice}'s belief regarding '{truth_question}'?",
        #     "question_type": "tom:belief:accessible:truth",
        #     "tom_type": "second-order",
        #     "correct_answer": f"{observer} believes that {accomplice} is aware of that.",
        #     "wrong_answer": f"{observer} is unaware that {accomplice}'s belief regarding that.",
        #     "truth_accessibility": "accessible"
        # })
    #######################################

    return {"beliefQAs": belief_qas}

In [69]:
print(json.dumps(generate_2ndbeliefQAs(data[0]), indent=4))

{
    "beliefQAs": [
        {
            "question": "What does Belinda believe about Ayra's belief regarding 'Who discussed 'Belinda and Leo want support Ayra's confidence and encourage Ayra to stay interested in their hobby even when Ayra is not good at it'?'?",
            "question_type": "tom:belief:accessible:reason",
            "tom_type": "second-order:AB",
            "correct_answer": "Belinda believes that Ayra is unaware that Belinda and Leo want support Ayra's confidence and encourage Ayra to stay interested in their hobby even when Ayra is not good at it, because Ayra was not involved in the conversation when this was discussed.",
            "wrong_answer": [
                "Belinda believes that Ayra believes that Belinda and Leo want support Ayra's confidence and encourage Ayra to stay interested in their hobby even when Ayra is not good at it.",
                "The text provides no information on what Belinda believes about Ayra's belief regarding that. Therefore

### 4. info_QAs

In [70]:
def generate_infoAccessibilityQAs(data):
    """
    生成 infoAccessibilityQA_list 和 infoAccessibilityQAs_binary
    """
    # 获取角色
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]
    
    # real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # truth = (data["lie"].get("truth_c") or "").rstrip(".")
    truth_id = data["truth_id"]
    real_reason = (data["lie"].get("real_reason_q") or data["lie"]["real_reason_c"]).rstrip(".")
    truth = data["lie"].get("truth_q") or data["lie"]["truth_c"].rstrip(".")
    lie = (data["lie"].get("lie_q") or data["lie"]["lie_c"]).rstrip(".")

    all_characters = [liar, accomplice, target, observer]
    # characters_binary = [accomplice, target, observer]
    if data.get("multiple_liar") is True:
        characters_binary = [target, observer]
    else:
        characters_binary = [accomplice, target, observer]
    correct_list_reason = [liar, accomplice]
    wrong_list_reason = [target, observer]

    # "information": f"{real_reason}? {liar} and {accomplice} discussed that.",
    # **生成 infoAccessibilityQA_list**
    # 之后model test怎么提供问题的？inofrmation:{inofrmation} + question:{question}
    info_access_list = [
        {
            "information": f"{real_reason}",
            "question": "List all the characters who know this information.",
            "question_type": "tom:info_accessibility:list:real_reason",
            "correct_answer": correct_list_reason,  # 正确答案是知道信息的角色
            "wrong_answer": wrong_list_reason,  # 错误答案是不知道信息的角色
            "real_reason_accessibility": "inaccessible"
        }
    ]
    

    # **如果 truth_id == 1，添加关于 truth 的信息可访问性问题**
    if truth_id == 1:
        correct_list_truth = [liar, accomplice, observer]
        wrong_list_truth = [target]

        info_access_list.append({
            "information": f"{truth}",
            "question": "List all the characters who know this information.",
            "question_type": "tom:info_accessibility:list:truth",
            "correct_answer": correct_list_truth,  # 正确答案是知道信息的角色
            "wrong_answer": wrong_list_truth,  # 错误答案是不知道信息的角色
            "missed_info_accessibility": "inaccessible"
        })
    

    # **生成 infoAccessibilityQAs_binary**
    info_access_binary = []
    
    # **关于 real_reason 的 binary question**
    for character in characters_binary:
        correct_answer = "yes" if character in correct_list_reason else "no"

        info_access_binary.append({
            "information": f"{real_reason}",
            "question": f"Does {character} know this information?",
            "question_type": "tom:info_accessibility:binary:real_reason",
            "correct_answer": correct_answer,
            "missed_info_accessibility": "inaccessible"
        })
    
    # **如果 truth_id == 1，添加关于 truth 的 binary question**
    if truth_id == 1:
        for character in characters_binary:
            correct_answer = "yes" if character in correct_list_truth else "no"

            info_access_binary.append({
                "information": f"{truth}",
                "question": f"Does {character} know this information?",
                "question_type": "tom:info_accessibility:binary:truth",
                "correct_answer": correct_answer,
                "missed_info_accessibility": "inaccessible"
            })

    return {
        "infoAccessibilityQA_list": info_access_list,  # 直接返回列表
        "infoAccessibilityQAs_binary": info_access_binary
    }

In [71]:
# if data[20].get("multiple_liar") is True:
#     characters_binary = 1
# else:
#     characters_binary = 2
# print(characters_binary)

In [72]:
print(json.dumps(generate_infoAccessibilityQAs(data[0]), indent=4))

{
    "infoAccessibilityQA_list": [
        {
            "information": "Belinda and Leo want support Ayra's confidence and encourage Ayra to stay interested in their hobby even when Ayra is not good at it",
            "question": "List all the characters who know this information.",
            "question_type": "tom:info_accessibility:list:real_reason",
            "correct_answer": [
                "Belinda",
                "Leo"
            ],
            "wrong_answer": [
                "Ayra",
                "Mia"
            ],
            "real_reason_accessibility": "inaccessible"
        },
        {
            "information": "Ayra is not talented for drawing at all",
            "question": "List all the characters who know this information.",
            "question_type": "tom:info_accessibility:list:truth",
            "correct_answer": [
                "Belinda",
                "Leo",
                "Mia"
            ],
            "wrong_answer": [
              

### 5. answerability_QAs

In [73]:
def generate_answerabilityQAs(data):
    """
    生成 answerabilityQA_list 和 answerabilityQAs_binary
    """
    # 获取角色
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]

    # real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # truth = data["lie"]["truth_c"].rstrip(".")
    # truth = (data["lie"].get("truth_c") or "").rstrip(".")
    real_reason = (data["lie"].get("real_reason_q") or data["lie"]["real_reason_c"]).rstrip(".")
    truth = data["lie"].get("truth_q") or data["lie"]["truth_c"].rstrip(".")
    lie = (data["lie"].get("lie_q") or data["lie"]["lie_c"]).rstrip(".")
    
    truth_id = data["truth_id"]

    characters_binary = [liar, target, accomplice] # 去掉了observer,跟multiple liar没关系

    correct_list_reason = [liar, accomplice]  # 知道 real_reason 的角色
    wrong_list_reason = [target, observer]

    correct_list_truth = [liar, accomplice, observer] if truth_id == 1 else []  # 知道 truth 的角色
    wrong_list_truth = [target]

    # **生成 answerabilityQA_list**
    answerability_list = [
        {
            "fact_question_real_reason": f"Who discussed '{real_reason}'?",
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:answerability:list:real_reason",
            "correct_answer": correct_list_reason,  # 知道 real_reason 的角色
            "wrong_answer": wrong_list_reason,  # 不知道 real_reason 的角色
            # "real_reason_accessibility": "inaccessible"
        }
    ]

    # **如果 truth_id == 1，添加关于 truth 的 answerability list question**
    if truth_id == 1:
        answerability_list.append({
            "fact_question_truth": f"Who discussed '{truth}'?",
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:answerability:list:truth",
            "correct_answer": correct_list_truth,  # 知道 truth 的角色
            "wrong_answer": wrong_list_truth,  # 不知道 truth 的角色
            # "truth_accessibility": "inaccessible"
        })

    # **生成 answerabilityQAs_binary**
    answerability_binary = []
    
    # **关于 real_reason 的 binary question**
    for character in characters_binary:
        correct_answer = "yes" if character in correct_list_reason else "no"

        answerability_binary.append({
            "fact_question_real_reason": f"Who discussed '{real_reason}'?",
            "question": f"Does {character} know the precise correct answer to this question?",
            "question_type": "tom:answerability:binary:real_reason",
            "correct_answer": correct_answer,
            # "real_reason_accessibility": "inaccessible"
        })
    
    
    # **如果 truth_id == 1，添加关于 truth 的 binary question**
    if truth_id == 1:
        for character in characters_binary:
            correct_answer = "yes" if character in correct_list_truth else "no"

            answerability_binary.append({
                "fact_question_truth": f"Who discussed '{truth}'?",
                "question": f"Does {character} know the precise correct answer to this question?",
                "question_type": "tom:answerability:binary:truth",
                "correct_answer": correct_answer,
                # "truth_accessibility": "inaccessible"
            })

    return {
        "answerabilityQA_list": answerability_list,  # 直接返回列表
        "answerabilityQAs_binary": answerability_binary
    }

In [74]:
print(json.dumps(generate_answerabilityQAs(data[0]), indent=4))

{
    "answerabilityQA_list": [
        {
            "fact_question_real_reason": "Who discussed 'Belinda and Leo want support Ayra's confidence and encourage Ayra to stay interested in their hobby even when Ayra is not good at it'?",
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:answerability:list:real_reason",
            "correct_answer": [
                "Belinda",
                "Leo"
            ],
            "wrong_answer": [
                "Ayra",
                "Mia"
            ]
        },
        {
            "fact_question_truth": "Who discussed 'Ayra is not talented for drawing at all'?",
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:answerability:list:truth",
            "correct_answer": [
                "Belinda",
                "Leo",
                "Mia"
            ],
     

### 6. liedetectability_QAs

In [75]:
def generate_liedetectabilityQAs(data):
    """
    生成 liedetectabilityQAs_list 和 liedetectabilityQAs_binary
    """
    # 获取角色
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]

    truth_id = data["truth_id"]
    
    real_reason = (data["lie"].get("real_reason_q") or data["lie"]["real_reason_c"]).rstrip(".")
    truth = data["lie"].get("truth_q") or data["lie"]["truth_c"].rstrip(".")
    lie = (data["lie"].get("lie_q") or data["lie"]["lie_c"]).rstrip(".")
    # lie = data["lie"]["lie_c"].rstrip(".")
    # real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # truth = (data["lie"].get("truth_c") or "").rstrip(".")
    falsification = bool(data["lie"]['falsification'])  # 确保 falsification 是布尔值

    all_characters = [liar, target, accomplice]
    correct_list_reason = [liar, accomplice]  # 知道 real_reason 的角色
    correct_list_truth = [liar, accomplice]  # 知道 truth 的角色, observer虽然知道truth但是不知道lie
    wrong_list_reason = [target]
    wrong_list_truth = [target]

    # **初始化列表**
    liedetectability_list = []

    if data.get("multiple_liar") is True:
        question = f"Was what {liar} and {accomplice} told {target}, '{lie}', true?"
    else:
        question = f"Was what {liar} told {target}, '{lie}', true?"

    # **生成 liedetectabilityQAs_list**
    if truth_id == 1 and falsification:  # 当 truth 存在
        liedetectability_list.append({
            # "fact_question_truth": f"Was what {liar} told {target}, '{lie}', true?",
            "comprehension_q": question,
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:liedetectability:list",
            "correct_answer": correct_list_truth,  # 包含 observer
            "wrong_answer": wrong_list_truth,  # 只有 target 不知道
        })
    else:
        liedetectability_list.append({
            "comprehension_q": question,
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:liedetectability:list",
            "correct_answer": correct_list_reason,  # 只有 liar 和 accomplice 知道
            "wrong_answer": wrong_list_reason,  # observer 和 target 不知道
        })

    # **生成 liedetectabilityQAs_binary**
    liedetectability_binary = []
    if truth_id == 1 and falsification:  # 当 truth 存在
        for character in all_characters:
            correct_answer = "yes" if character in correct_list_truth else "no"

            liedetectability_binary.append({
                # "comprehension_q": f"Was what {liar} told {target}, '{lie}', true?",
                "comprehension_q": question,
                "question": f"Does {character} know the precise correct answer to this question?",
                "question_type": "tom:liedetectability:binary",
                "correct_answer": correct_answer,
            })
    else:
        for character in all_characters:
            correct_answer = "yes" if character in correct_list_reason else "no"
            liedetectability_binary.append({
                # "comprehension_q": f"Was what {liar} told {target}, '{lie}', true?",
                "comprehension_q": question,
                "question": f"Does {character} know the precise correct answer to this question?",
                "question_type": "tom:liedetectability:binary",
                "correct_answer": correct_answer,
            })

    return {
        "liedetectabilityQAs_list": liedetectability_list,  # 直接返回列表
        "liedetectabilityQAs_binary": liedetectability_binary
    }

In [78]:
print(json.dumps(generate_liedetectabilityQAs(data[0]), indent=4))

{
    "liedetectabilityQAs_list": [
        {
            "comprehension_q": "Was what Belinda and Leo told Ayra, 'Ayra's drawing is really creative and Ayra is getting very talented', true?",
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:liedetectability:list",
            "correct_answer": [
                "Belinda",
                "Leo"
            ],
            "wrong_answer": [
                "Ayra"
            ]
        }
    ],
    "liedetectabilityQAs_binary": [
        {
            "comprehension_q": "Was what Belinda and Leo told Ayra, 'Ayra's drawing is really creative and Ayra is getting very talented', true?",
            "question": "Does Belinda know the precise correct answer to this question?",
            "question_type": "tom:liedetectability:binary",
            "correct_answer": "yes"
        },
        {
            "comprehension_q": "Was what Belinda and Leo told Ay

### 7. lieability_QAs

In [79]:
def generate_lieabilityQAs(data):
    """
    生成 lieability
    """
    # 获取角色
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]

    # lie = data["lie"]["lie_c"].rstrip(".")
    # real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # truth = (data["lie"].get("truth_c") or "").rstrip(".")
    
    truth_id = data["truth_id"]

    real_reason = (data["lie"].get("real_reason_q") or data["lie"]["real_reason_c"]).rstrip(".")
    truth = data["lie"].get("truth_q") or data["lie"]["truth_c"].rstrip(".")
    lie = (data["lie"].get("lie_q") or data["lie"]["lie_c"]).rstrip(".")

    lieability = []

    if truth_id == 1:
        # +-
        first_wrong_option_truth_liar = f"{liar} tell {target} that {lie} because {liar} believes that {target} believes {truth}."
        # -+
        second_wrong_option_truth_liar = f"{liar} tell {target} that {lie}' because {liar} is unaware that {target} is unaware that {truth}."
        # --
        third_wrong_option_truth_liar = f"{liar} tell {target} that {lie}' because {liar} is unaware that {target} believes {truth}."

        # First question: Explanation based on truth
        lieability.append({
            # "fact_question_truth": f"Was what {liar} told {target}, '{lie}', true?",
            "question": f"Please choose the correct answer from the options below:",
            "question_type": "tom:lieability:binary:liar",
            "correct_answer": f"{liar} tell {target} that '{lie}' because {liar} believes that {target} is unaware that {truth}.", #++
            "wrong_answer": [
                first_wrong_option_truth_liar,
                second_wrong_option_truth_liar,
                third_wrong_option_truth_liar
            ],
        }),

    # Second question: Explanation based on real_reason
    # +-
    first_wrong_option_real_reason = f"{liar} tell {target} that {lie}' because {liar} believes that {target} believes {real_reason}."
    # -+
    second_wrong_option_real_reason = f"{liar} tell {target} that {lie}' because {liar} is unaware that {target} is unaware that {real_reason}."
    # --
    third_wrong_option_real_reason = f"{liar} tell {target} that {lie}' because {liar} is unaware that {target} believes {real_reason}."

    lieability.append({
        # "fact_question_truth": f"Was what {liar} told {target}, '{lie}', true?",
        "question": f"Please choose the correct answer from the options below:",
        "question_type": "tom:lieability:mcq:liar",
        "correct_answer": f"{liar} tell {target} that '{lie}' because {liar} believes that {target} is unaware that {real_reason}.",
        "wrong_answer": [
            first_wrong_option_real_reason,
            second_wrong_option_real_reason,
            third_wrong_option_real_reason
        ],
    })

    # # 多个liar的时候加上一些question：accompliace also is liar now
    if data.get("multiple_liar") is True:
        # Second question: Explanation based on real_reason
        # +-
        first_wrong_option_real_reason_accomplice = f"{accomplice} tell {target} that '{lie}'' because {accomplice} believes that {target} believes {real_reason}."
        # -+
        second_wrong_option_real_reason_accomplice = f"{accomplice} tell {target} that '{lie}' because {accomplice} is unaware that {target} is unaware that {real_reason}."
        # --
        third_wrong_option_real_reason_accomplice = f"{accomplice} tell {target} that '{lie}' because {accomplice} is unaware that {target} believes {real_reason}."

        lieability.append({
        # "fact_question_truth": f"Was what {liar} told {target}, '{lie}', true?",
        "question": f"Please choose the correct answer from the options below:",
        "question_type": "tom:lieability:mcq:accomplice",
        "correct_answer": f"{accomplice} tell {target} that '{lie}' because {accomplice} believes that {target} is unaware that {real_reason}.",
        "wrong_answer": [
            first_wrong_option_real_reason_accomplice,
            second_wrong_option_real_reason_accomplice,
            third_wrong_option_real_reason_accomplice
        ],
    })
        if truth_id == 1:
            # +-
            first_wrong_option_truth_accomplice = f"{accomplice} tell {target} that '{lie}' because {accomplice} believes that {target} believes {truth}."
            # -+
            second_wrong_option_truth_accomplice = f"{accomplice} tell {target} that '{lie}' because {accomplice} is unaware that {target} is unaware that {truth}."
            # --
            third_wrong_option_truth_accomplice = f"{accomplice} tell {target} that '{lie}' because {accomplice} is unaware that {target} believes {truth}."

            lieability.append({
            # "fact_question_truth": f"Was what {liar} told {target}, '{lie}', true?",
            "question": f"Please choose the correct answer from the options below:",
            "question_type": "tom:lieability:mcq:accomplice",
            "correct_answer": f"{accomplice} tell {target} that '{lie}' because {accomplice} believes that {target} is unaware that {truth}.", #++
            "wrong_answer": [
                first_wrong_option_truth_accomplice,
                second_wrong_option_truth_accomplice,
                third_wrong_option_truth_accomplice
            ],
        })        
        
    return {
        "lieabilityQAs": lieability,
    }

In [80]:
print(json.dumps(generate_lieabilityQAs(data[0]), indent=4))

{
    "lieabilityQAs": [
        {
            "question": "Please choose the correct answer from the options below:",
            "question_type": "tom:lieability:binary:liar",
            "correct_answer": "Belinda tell Ayra that 'Ayra's drawing is really creative and Ayra is getting very talented' because Belinda believes that Ayra is unaware that Ayra is not talented for drawing at all.",
            "wrong_answer": [
                "Belinda tell Ayra that Ayra's drawing is really creative and Ayra is getting very talented because Belinda believes that Ayra believes Ayra is not talented for drawing at all.",
                "Belinda tell Ayra that Ayra's drawing is really creative and Ayra is getting very talented' because Belinda is unaware that Ayra is unaware that Ayra is not talented for drawing at all.",
                "Belinda tell Ayra that Ayra's drawing is really creative and Ayra is getting very talented' because Belinda is unaware that Ayra believes Ayra is not talente

### 8. put into jsonl

In [36]:
def merge_beliefQAs(d1, d2):
    """
    d1 和 d2 都是形如 {"beliefQAs": [...]} 的字典
    将 d1 和 d2 的 "beliefQAs" 列表合并到同一个字典中
    """
    # 如果你想要新建一个字典，不修改原对象，可以这样：
    return {
        "beliefQAs": d1["beliefQAs"] + d2["beliefQAs"]
    }

In [37]:
def assign_question_ids(new_entry: dict, set_id: str) -> dict:
    """
    给 new_entry 中的每个问题加上 q_id。
    假设以下字段都存储为 list：
      - beliefQAs
      - infoAccessibilityQA_list
      - infoAccessibilityQAs_binary
      - answerabilityQA_list
      - answerabilityQAs_binary
      - liedetectabilityQAs_list
      - liedetectabilityQAs_binary
      - lieabilityQAs
    并且你希望每个问题都能分配到一个唯一的 q_id。

    参数:
        new_entry (dict): 包含多个 QA 列表的字典。
        set_id (str): 用作 q_id 前缀 (例如 "0-0-0")

    返回:
        dict: 在原字典基础上修改并返回，给每条问题都添加了 "q_id" 字段
    """
    # comprehensionQA
    if "comprehensionQA" in new_entry:
        for i, q_dict in enumerate(new_entry["comprehensionQA"]):
            q_dict["q_id"] = f"{set_id}-comprehension-{i}"

    # justificationQA
    if "justificationQA" in new_entry:
        for i, q_dict in enumerate(new_entry["justificationQA"]):
            q_dict["q_id"] = f"{set_id}-justification-{i}"


    # reasonQA
    if "fact_reasonQA" in new_entry:
        for i, q_dict in enumerate(new_entry["fact_reasonQA"]):
            q_dict["q_id"] = f"{set_id}-fact_reason-{i}"

    # truthQA
    if "fact_truthQA" in new_entry:
        for i, q_dict in enumerate(new_entry["fact_truthQA"]):
            q_dict["q_id"] = f"{set_id}-fact_truth-{i}"

    # beliefQAs
    if "beliefQAs" in new_entry:
        for i, q_dict in enumerate(new_entry["beliefQAs"]):
            q_dict["q_id"] = f"{set_id}-belief-{i}"

    # infoAccessibilityQA_list
    if "infoAccessibilityQA_list" in new_entry:
        for i, q_dict in enumerate(new_entry["infoAccessibilityQA_list"]):
            q_dict["q_id"] = f"{set_id}-info_access_list-{i}"

    # infoAccessibilityQAs_binary
    if "infoAccessibilityQAs_binary" in new_entry:
        for i, q_dict in enumerate(new_entry["infoAccessibilityQAs_binary"]):
            q_dict["q_id"] = f"{set_id}-info_access_binary-{i}"

    # answerabilityQA_list
    if "answerabilityQA_list" in new_entry:
        for i, q_dict in enumerate(new_entry["answerabilityQA_list"]):
            q_dict["q_id"] = f"{set_id}-answerability_list-{i}"

    # answerabilityQAs_binary
    if "answerabilityQAs_binary" in new_entry:
        for i, q_dict in enumerate(new_entry["answerabilityQAs_binary"]):
            q_dict["q_id"] = f"{set_id}-answerability_binary-{i}"

    # liedetectabilityQAs_list
    if "liedetectabilityQAs_list" in new_entry:
        for i, q_dict in enumerate(new_entry["liedetectabilityQAs_list"]):
            q_dict["q_id"] = f"{set_id}-liedetect_list-{i}"

    # liedetectabilityQAs_binary
    if "liedetectabilityQAs_binary" in new_entry:
        for i, q_dict in enumerate(new_entry["liedetectabilityQAs_binary"]):
            q_dict["q_id"] = f"{set_id}-liedetect_binary-{i}"
    
    # lieabilityQAs
    if "lieabilityQAs" in new_entry:
        for i, q_dict in enumerate(new_entry["lieabilityQAs"]):
            q_dict["q_id"] = f"{set_id}-lieability-{i}"

    return new_entry

In [85]:
with open("/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_element_3.json", "r", encoding="utf-8") as file:
    content = file.read().strip()
    if content == "":
        data = []  # 注意：这里是 list，因为是多个 set 的列表
    else:
        data = json.loads(content)

In [4]:
# # 1. 可以手动指定要处理的 set_id，例如 "0-0-0"
# selected_set_id = "0-1-0-2"

# # 选择要处理的 set
# selected_set = None

# for item in data:
#     if item["set_id"] == selected_set_id:
#         selected_set = item
#         break
# if selected_set is None:
#     print(f"Set ID {selected_set_id} not found.")
# else:
#     print(f"Processing set: {selected_set_id}")

In [86]:
# 2. 循环遍历所有 set
for item in data:
    # 1. 把当前条目直接赋值给 selected_set
    selected_set = item
    # 2. 把其中的 set_id 取出来，赋值给 selected_set_id
    selected_set_id = item.get("set_id", "unknown")

    print(f"Processing set: {selected_set_id}")

    # **生成问题** 遍历所有的问题generation函数
    comprehension_qas = generate_comprehensionQA(selected_set)
    justification_qas = generate_justificationQA(selected_set)

    fact_qas = generate_fact_QA(selected_set)

    belief_qas_1 = generate_1stbeliefQAs(selected_set)
    belief_qas_2 = generate_2ndbeliefQAs(selected_set)
    belief_qas = merge_beliefQAs(belief_qas_1, belief_qas_2)

    info_accessibility_qas = generate_infoAccessibilityQAs(selected_set)
    answerability_qas = generate_answerabilityQAs(selected_set)
    
    liedetectability_qas = generate_liedetectabilityQAs(selected_set)  # ✅ 新增的 Lie Detectability 逻辑

    lieability_qas = generate_lieabilityQAs(selected_set)  # ✅ 新增的 Lie Detectability 逻辑

    # 如果只想保留原始set中的一部分信息
    selected_fields = {key: selected_set[key] for key in ["set_id", "lie_id", "conv_id", "truth", "lie_type", "emotion"] if key in selected_set}
    
    # **合并生成的数据**
    new_entry = {
        **selected_set,  # 复制原始数据
        **comprehension_qas,
        **justification_qas,
        **fact_qas,
        **belief_qas,
        **info_accessibility_qas,
        **answerability_qas,
        **liedetectability_qas,  # ✅ 加入 Lie Detectability QAs
        **lieability_qas,  # ✅ 加入 Lie Detectability QAs
    }
    
    # 调用assign_question_ids给每条 reasonQA / truthQA 的问题添加唯一的 q_id
    new_entry = assign_question_ids(new_entry, set_id=selected_set_id)

    # **写入 `whitetom_set.json`**
    output_filename = "/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_3.json"

    # 读取 JSON 文件（如果文件不存在，则创建一个空列表）
    try:
        with open(output_filename, "r", encoding="utf-8") as file:
            existing_data = json.load(file)  # 读取已有数据
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading JSON (likely file not found or empty): {e}")
        existing_data = []  # 如果文件不存在，则初始化为空列表

    # **检查是否已有该 `set_id`，如果有就覆盖**
    found = False
    for i, entry in enumerate(existing_data):
        if entry["set_id"] == selected_set_id:
            existing_data[i] = new_entry  # **覆盖旧数据**
            found = True
            break

    if not found:
        existing_data.append(new_entry)  # **如果没有，才追加**

    # **写回文件**
    with open(output_filename, "w", encoding="utf-8") as file:
        json.dump(existing_data, file, indent=4, ensure_ascii=False)

    print(f"Data for set {selected_set_id} written to {output_filename}")

    # **立即检查是否写入成功**
    with open(output_filename, "r", encoding="utf-8") as file:
        content = file.read()
        print("Written JSON Content:\n", content)  # 检查是否写入了内容

Processing set: 3-14-0-1
Error loading JSON (likely file not found or empty): [Errno 2] No such file or directory: '/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_3.json'
Data for set 3-14-0-1 written to /Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_3.json
Written JSON Content:
 [
    {
        "set_id": "3-14-0-1",
        "lie_id": "0-1",
        "conv_id": 14,
        "truth_id": 1,
        "lie_type": "altruistic_white_lies",
        "emotion": "sad",
        "topic": {
            "scenario": "chatting casually about Ayra's hobbies and recent school life",
            "situation_topic": "Ayra is taking painting courses recently",
            "situation": "Ayra is excited to share their new drawing with the family and asks for feedback",
            "lie_objective": "Ayraoost Ayra's confidence and enthusiasm",
            "leave_reason_B": "feed the cat",
            "leave_reason_D_1": "have to quickly arrange something for a later meeting",
            

In [92]:
import json

def replace_data_by_set_id(
    path_original="/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_0.json",
    path_replacement="/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_0-1.json",
    path_output="/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_0_updated.json"
):
    """
    将 Tactful_conv_set_0-1.json 中的
    1. infoAccessibilityQA_list
    2. lieabilityQAs
    3. justificationQAs (或 justification_question 等自定义字段)
    通过 set_id 匹配，批量替换到 Tactful_conv_set_0.json 对应条目中
    """

    # 1. 读取原始数据
    with open(path_original, "r", encoding="utf-8") as f:
        data_original = json.load(f)

    # 2. 读取要替换的文件数据
    with open(path_replacement, "r", encoding="utf-8") as f:
        data_replacement = json.load(f)

    # 3. 分别建立三个映射字典
    #    {set_id: infoAccessibilityQA_list}、{set_id: lieabilityQAs}、{set_id: justificationQAs}
    infoAccessibility_map = {}
    lieability_map = {}
    justification_map = {}  # 用于存储 justificationQAs

    for item in data_replacement:
        set_id = item.get("set_id")
        if set_id is not None:
            # 如果有 infoAccessibilityQA_list，就放到 infoAccessibility_map
            if "infoAccessibilityQA_list" in item:
                infoAccessibility_map[set_id] = item["infoAccessibilityQA_list"]
            # 如果有 lieabilityQAs，就放到 lieability_map
            if "lieabilityQAs" in item:
                lieability_map[set_id] = item["lieabilityQAs"]
            # 如果有 justificationQAs，就放到 justification_map
            # (若你的字段名不同，如 "justification_question"，请对应修改)
            if "justificationQAs" in item:
                justification_map[set_id] = item["justificationQAs"]

    # 4. 遍历原始数据，根据 set_id 替换对应字段
    for item in data_original:
        set_id = item.get("set_id")
        if set_id in infoAccessibility_map:
            item["infoAccessibilityQA_list"] = infoAccessibility_map[set_id]
        if set_id in lieability_map:
            item["lieabilityQAs"] = lieability_map[set_id]
        if set_id in justification_map:
            item["justificationQAs"] = justification_map[set_id]

    # 5. 将更新后的数据写入新的 JSON 文件
    with open(path_output, "w", encoding="utf-8") as f:
        json.dump(data_original, f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    replace_data_by_set_id()
    print("infoAccessibilityQA_list & lieabilityQAs & justificationQAs 替换完成！")

infoAccessibilityQA_list & lieabilityQAs & justificationQAs 替换完成！
